In [1]:
import os
from pymatgen.io.vasp import Poscar
from pymatgen.io.cif import CifWriter

# Input and output folders
pos_folder = "best_poscars"
cif_folder = "cif"

# Make sure output folder exists
os.makedirs(cif_folder, exist_ok=True)

# Loop through all files in the input folder
for filename in os.listdir(pos_folder):
    if "POSCAR" in filename:  # matches POSCAR, POSCAR_1, 1_POSCAR-0.125, etc
        poscar_path = os.path.join(pos_folder, filename)
        try:
            poscar = Poscar.from_file(poscar_path)
            structure = poscar.structure

            # Build CIF filename
            cif_name = os.path.splitext(filename)[0] + ".cif"
            cif_path = os.path.join(cif_folder, cif_name)

            # Write CIF
            CifWriter(structure).write_file(cif_path)
            print(f"Converted: {filename} → {cif_name}")
        except Exception as e:
            print(f"Failed for {filename}: {e}")


Converted: 2_POSCAR-0.25 → 2_POSCAR-0.cif
Converted: 5_POSCAR-0.625 → 5_POSCAR-0.cif
Converted: 1_POSCAR-0.125 → 1_POSCAR-0.cif
Converted: 3_POSCAR-0.375 → 3_POSCAR-0.cif
Converted: 4_POSCAR-0.5 → 4_POSCAR-0.cif
Converted: 6_POSCAR-0.75 → 6_POSCAR-0.cif
Converted: 7_POSCAR-0.875 → 7_POSCAR-0.cif


In [2]:
import os
from pymatgen.io.vasp import Poscar

pos_folder = "best_poscars"

for filename in os.listdir(pos_folder):
    if "POSCAR" in filename:
        poscar_path = os.path.join(pos_folder, filename)
        poscar = Poscar.from_file(poscar_path)
        structure = poscar.structure

        comp = structure.composition
        comp = comp.fractional_composition  # normalize

        # Extract fractions (adjust names to your system)
        mn = comp.get("Mn", 0)
        zn = comp.get("Zn", 0)
        fe = comp.get("Fe", 0)
        o = comp.get("O", 0)

        # Normalize to O = 4
        factor = 4 / o
        mn *= factor
        zn *= factor
        fe *= factor
        o *= factor

        # Build custom formula string
        formula = f"Mn{mn:.3f}Zn{zn:.3f}(FeO2)2"

        print(f"{filename} → {formula}")


2_POSCAR-0.25 → Mn0.750Zn0.250(FeO2)2
5_POSCAR-0.625 → Mn0.375Zn0.625(FeO2)2
1_POSCAR-0.125 → Mn0.875Zn0.125(FeO2)2
3_POSCAR-0.375 → Mn0.625Zn0.375(FeO2)2
4_POSCAR-0.5 → Mn0.500Zn0.500(FeO2)2
6_POSCAR-0.75 → Mn0.250Zn0.750(FeO2)2
7_POSCAR-0.875 → Mn0.125Zn0.875(FeO2)2


In [3]:
import os
from pymatgen.io.vasp import Poscar
from pymatgen.analysis.diffraction.xrd import XRDCalculator
import matplotlib.pyplot as plt

# Input folder with POSCARs
pos_folder = "best_poscars"
xrd_folder = "xrd_patterns"
os.makedirs(xrd_folder, exist_ok=True)

# Setup XRD calculator (Cu Kα by default, λ = 1.5406 Å)
xrd_calc = XRDCalculator(wavelength="CuKa")

for filename in os.listdir(pos_folder):
    if "POSCAR" in filename:
        poscar_path = os.path.join(pos_folder, filename)
        poscar = Poscar.from_file(poscar_path)
        structure = poscar.structure

        # --- Get composition & custom formula ---
        comp = structure.composition.fractional_composition
        mn = comp.get("Mn", 0)
        zn = comp.get("Zn", 0)
        la = comp.get("La", 0)
        fe = comp.get("Fe", 0)
        o = comp.get("O", 0)

        # Normalize to O = 4
        factor = 4 / o
        mn *= factor
        zn *= factor
        la *= factor
        fe *= factor
        o *= factor
        formula = f"Mn{mn:.3f}Zn{zn:.3f}La{la:.3f}(FeO2)2"

        # --- Simulate XRD ---
        pattern = xrd_calc.get_pattern(structure, two_theta_range=(10, 80))

        # --- Plot ---
        plt.figure(figsize=(8,4))

        # Newer matplotlib has use_line_collection, older doesn't → safe way:
        markerline, stemlines, baseline = plt.stem(pattern.x, pattern.y, basefmt=" ")
        plt.setp(stemlines, 'linewidth', 1.2)   # adjust line width
        plt.setp(markerline, 'marker', ' ', 'color', 'k')  # no markers

        plt.xlabel("2θ (degrees)")
        plt.ylabel("Intensity (a.u.)")
        plt.title(f"Simulated XRD: {formula}")
        plt.tight_layout()

        # Save figure
        xrd_path = os.path.join(
            xrd_folder, f"{os.path.splitext(filename)[0]}_{formula}_XRD.png"
        )
        plt.savefig(xrd_path, dpi=300)
        plt.close()

        print(f"Simulated XRD saved: {xrd_path}")


Simulated XRD saved: xrd_patterns/2_POSCAR-0_Mn0.750Zn0.250La0.000(FeO2)2_XRD.png
Simulated XRD saved: xrd_patterns/5_POSCAR-0_Mn0.375Zn0.625La0.000(FeO2)2_XRD.png
Simulated XRD saved: xrd_patterns/1_POSCAR-0_Mn0.875Zn0.125La0.000(FeO2)2_XRD.png
Simulated XRD saved: xrd_patterns/3_POSCAR-0_Mn0.625Zn0.375La0.000(FeO2)2_XRD.png
Simulated XRD saved: xrd_patterns/4_POSCAR-0_Mn0.500Zn0.500La0.000(FeO2)2_XRD.png
Simulated XRD saved: xrd_patterns/6_POSCAR-0_Mn0.250Zn0.750La0.000(FeO2)2_XRD.png
Simulated XRD saved: xrd_patterns/7_POSCAR-0_Mn0.125Zn0.875La0.000(FeO2)2_XRD.png


In [6]:
import os
from pymatgen.io.vasp import Poscar
from pymatgen.analysis.diffraction.xrd import XRDCalculator
import matplotlib.pyplot as plt
import itertools

# Input folder with POSCARs
pos_folder = "best_poscars"
xrd_folder = "xrd_patterns"
os.makedirs(xrd_folder, exist_ok=True)

# Setup XRD calculator (Cu Kα by default, λ = 1.5406 Å)
xrd_calc = XRDCalculator(wavelength="CuKa")

# Prepare figure for overlapping patterns
plt.figure(figsize=(10, 6))
colors = itertools.cycle(plt.cm.tab10.colors)  # cycle through colors

for filename in os.listdir(pos_folder):
    if "POSCAR" in filename:
        poscar_path = os.path.join(pos_folder, filename)
        poscar = Poscar.from_file(poscar_path)
        structure = poscar.structure

        # --- Get composition & custom formula ---
        comp = structure.composition.fractional_composition
        mn = comp.get("Mn", 0)
        zn = comp.get("Zn", 0)
        fe = comp.get("Fe", 0)
        o = comp.get("O", 0)

        # Normalize to O = 4
        factor = 4 / o
        mn *= factor
        zn *= factor
        fe *= factor
        o *= factor
        formula = f"Mn{mn:.3f}Zn{zn:.3f}(FeO2)2"

        # --- Simulate XRD ---
        pattern = xrd_calc.get_pattern(structure, two_theta_range=(10, 80))

        # --- Plot (overlapping) ---
        color = next(colors)
        markerline, stemlines, baseline = plt.stem(pattern.x, pattern.y, basefmt=" ")
        plt.setp(stemlines, "color", color, "linewidth", 1.2)
        plt.setp(markerline, "marker", " ", "color", color)
        baseline.set_color("k")  # baseline color
        plt.plot([], [], color=color, label=formula)  # dummy plot for legend

plt.xlabel("2θ (degrees)")
plt.ylabel("Intensity (a.u.)")
plt.title("Simulated XRD Patterns (Overlapped)")
plt.legend(fontsize=8)
plt.tight_layout()

# Save combined figure
xrd_path = os.path.join(xrd_folder, "overlapped_XRD.png")
plt.savefig(xrd_path, dpi=300) 
plt.close()
print(f"Overlapped XRD saved: {xrd_path}")

Overlapped XRD saved: xrd_patterns/overlapped_XRD.png
